In [11]:
import numpy as np
import random

Beta = 0.1
ePrev = 1
eNew = 1
print(min(1, np.exp(-Beta*(ePrev-eNew))))


def accept_move(ePrev,eNew,beta):
  proba = min(1, np.exp(-Beta*(ePrev-eNew)))
  return random.random() < proba



accept_move(ePrev,eNew,Beta)

1


True

In [26]:
8 if False else 4

4

In [23]:
test_pos = np.array([3,6,5])
maxs = np.array([4,5,6])
mins = [0,0,0]

print(np.any((test_pos > maxs)))

True


In [15]:
np.any([1,3,4] > maxs)

False

In [31]:
a = np.array([1,None,3])
a = None

if np.all(a) != None:
  print("yes")
else:
  print("False")

False
